In [286]:
!pip install selenium
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,231 kB]
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main amd64 Packages [1,145 kB]
Fetched 3,736 k

In [287]:
import csv
import numpy as np
import pandas as pd
from google.colab import files
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [288]:
def web_driver():
  options = webdriver.ChromeOptions()
  options.add_argument('--verbose')
  options.add_argument('--no-sandbox')
  options.add_argument('--headless')
  options.add_argument('--disable-gpu')
  options.add_argument("--window-size-1920, 1200")
  options.add_argument('--disable-dev-shm-usage')
  # Khai báo trình duyệt
  driver = webdriver.Chrome(options=options)
  return driver

driver = web_driver()
data = []

In [289]:
all_car_links = []
for page in range(1, 30):
    url = f"https://xe.chotot.com/mua-ban-oto?f=protection_entitlement?protection_entitlement=true&page={page}"
    driver.get(url)


    # Thu thập liên kết của xe và lọc ra những link của người đăng bán
    elemsCars = driver.find_elements(By.CSS_SELECTOR, ".AdItem_wrapperAdItem__S6qPH [href]")
    elemsNotCars = driver.find_elements(By.CSS_SELECTOR, ".AdItem_adItem__gDDQT [href]")
    linksCars = [elem.get_attribute('href') for elem in elemsCars]
    linksNotCars = [elem.get_attribute('href') for elem in elemsNotCars]

    # Lọc ra những đường liên kết chỉ thuộc về elemsCars và không trùng với elemsNotCars
    uniqueLinks = list(set(linksCars) - set(linksNotCars))

    all_car_links.extend(uniqueLinks)


    for link in all_car_links:
        print(link)

Streaming output truncated to the last 5000 lines.
https://xe.chotot.com/mua-ban-oto-quan-long-bien-ha-noi/111410020.htm
https://xe.chotot.com/mua-ban-oto-quan-nam-tu-liem-ha-noi/109567760.htm
https://xe.chotot.com/mua-ban-oto-quan-8-tp-ho-chi-minh/109221469.htm
https://xe.chotot.com/mua-ban-oto-huyen-binh-chanh-tp-ho-chi-minh/110126674.htm
https://xe.chotot.com/mua-ban-oto-quan-hoang-mai-ha-noi/111363828.htm#px=SR-stickyad-[PO-5][PL-top]
https://xe.chotot.com/mua-ban-oto-quan-binh-tan-tp-ho-chi-minh/101767718.htm#px=SR-stickyad-[PO-2][PL-top]
https://xe.chotot.com/mua-ban-oto-thanh-pho-thu-duc-tp-ho-chi-minh/107107468.htm#px=SR-special_display_ad-[PO-16][PL-default]
https://xe.chotot.com/mua-ban-oto-quan-thanh-xuan-ha-noi/111421641.htm
https://xe.chotot.com/mua-ban-oto-quan-thanh-xuan-ha-noi/109607022.htm
https://xe.chotot.com/mua-ban-oto-quan-7-tp-ho-chi-minh/110532274.htm
https://xe.chotot.com/mua-ban-oto-quan-10-tp-ho-chi-minh/111056828.htm#px=SR-special_display_ad-[PO-6][PL-defaul

Tên xe đăng bán, giá bán, Tên người bán, Hãng, Dòng xe, Năm sản xuất, Số Km đã đi, Hộp số, Tình trạng, Nhiên liệu, Xuất xứ, Kiểu dáng, Số chỗ, Trọng lượng, Chính sách bảo hành, Trọng tải

In [290]:
print("Số lượng liên kết cần trích xuất:", len(all_car_links))


Số lượng liên kết cần trích xuất: 590


In [291]:
import csv
import pandas as pd

# Mở một file CSV để ghi
with open('car_info.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = ['Tên đăng bán', 'Giá bán', 'Hãng', 'Dòng xe', 'Năm sản xuất', 'Số Km đã đi',
                  'Tình trạng', 'Hộp số', 'Nhiên liệu',
                  'Xuất xứ', 'Kiểu dáng', 'Số chỗ', 'Chính sách bảo hành',
                  'Trọng lượng', 'Trọng tải']  # Thêm cột mới

    writer = csv.writer(csvfile, delimiter=';')  # Sử dụng dấu ';' làm delimiter

    # Viết tiêu đề của các cột vào file CSV
    writer.writerow(fieldnames)

    for link in all_car_links:
        driver.get(link)

        # Lấy tên đăng bán xe và giá bán
        titles = driver.find_element(By.CSS_SELECTOR, ".AdDecriptionVeh_adTitle__vEuKD")
        title = titles.text
        elems_price = driver.find_element(By.CSS_SELECTOR, ".AdDecriptionVeh_price__u_N83")
        price = elems_price.text

        # Lấy thông tin chi tiết về xe
        # Nhấp vào nút để xem thông tin chi tiết
        button = driver.find_element(By.CSS_SELECTOR, '.styles_button__SVZnw')
        driver.execute_script("arguments[0].click();", button)
        car_elems = driver.find_elements(By.CSS_SELECTOR, ".AdParam_adParamItemVeh__mCpPS")
        car_info = [elem.text for elem in car_elems]

        # Tạo một từ điển tạm thời để xác định xem trường nào đã được thêm
        car_temp_data = {}

        # Thêm thông tin chi tiết vào từ điển tạm thời
        for info in car_info:
            key, value = info.split(":", 1)
            key = key.strip()
            value = value.strip()
            car_temp_data[key] = value

        # Tạo một danh sách chứa thông tin của mỗi xe
        car_data = []

        # Duyệt qua danh sách cột và thêm giá trị hoặc '?' nếu cột không tồn tại
        for field in fieldnames:
            if field in car_temp_data:
                car_data.append(car_temp_data[field])
            elif field == 'Tên đăng bán':
                car_data.append(title)  # Thêm tên đăng bán xe
            elif field == 'Giá bán':
                car_data.append(price)  # Thêm giá bán
            else:
                car_data.append('?')

        # Ghi thông tin xe vào file CSV
        writer.writerow(car_data)

# Đóng trình duyệt sau khi đã xong
driver.quit()

# Sau khi bạn đã ghi dữ liệu vào file CSV, bạn có thể đọc nó vào DataFrame Pandas như sau:
columns = fieldnames  # Sử dụng các cột bạn đã định nghĩa
data = pd.read_csv('car_info.csv', delimiter=';', names=columns)


In [293]:
data

,Tên đăng bán,Giá bán,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Chính sách bảo hành,Trọng lượng,Trọng tải
0,Tên đăng bán,Giá bán,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Chính sách bảo hành,Trọng lượng,Trọng tải
1,Mazda cx5 2.0 2021 màu đỏ bản Premium,755.000.000 đ,Mazda,CX 5,2021,38000,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,5,Bảo hành hãng,> 1 tấn,> 2 tấn
2,💥Mercedes c200 sx 2017 2.0 AT cao cấp od 6v km 💥,785.000.000 đ,Mercedes Benz,C Class,2017,60000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn
3,XE Range Rover HSE lên Model 2020 bản 5 ghế,1.999.000.000 đ,LandRover,Range Rover,2013,2,Đã sử dụng,Tự động,Xăng,Nước khác,SUV / Cross over,8,Bảo hành hãng,> 1 tấn,> 2 tấn
4,HYUNHDAI PALIDASE 2023 - DÒNG SUV CỰC CHẤT,1.469.000.000 đ,Hyundai,Palisade,2023,?,Mới,Tự động,Xăng,Việt Nam,SUV / Cross over,7,Bảo hành hãng,> 1 tấn,> 2 tấn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,Hyundai Accent 2022 bản đủ ATH biển Hà Nội 1 chủ,485.000.000 đ,Hyundai,Accent,2022,20000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,2,Bảo hành hãng,> 1 tấn,> 2 tấn
587,ban Suzuki xl 7 sx 2022,529.000.000 đ,Suzuki,XL 7,2022,30000,Đã sử dụng,Tự động,Xăng,Ấn Độ,Minivan (MPV),7,Bảo hành hãng,> 1 tấn,> 2 tấn
588,Toyota Fortuner 2013 máy xăng số tự động,435.000.000 đ,Toyota,Fortuner,2013,110000,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,7,Bảo hành hãng,> 1 tấn,> 2 tấn
589,Vinfast Fadil bản plus sx cuối 2020 Full đồ,340.000.000 đ,Vinfast,Fadil,2020,24256,Đã sử dụng,Tự động,Xăng,Việt Nam,Hatchback,5,Bảo hành hãng,> 1 tấn,> 2 tấn


In [307]:
# Tải file .csv về máy tính
files.download('car_info.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [311]:
import pandas as pd

# Đọc dữ liệu từ URL
url = "/content/car_info (10).csv"
columns = ['Tên đăng bán', 'Giá bán', 'Hãng', 'Dòng xe', 'Năm sản xuất', 'Số Km đã đi',
                  'Tình trạng', 'Hộp số', 'Nhiên liệu',
                  'Xuất xứ', 'Kiểu dáng', 'Số chỗ', 'Chính sách bảo hành',
                  'Trọng lượng', 'Trọng tải']
data = pd.read_csv(url, sep=";", names=columns)
data = data.replace('?', pd.NA)
null_values = data[columns].isnull()

# Đếm số lượng giá trị True (tức là giá trị null)
count_null_values = null_values.sum()

print(f"Số giá trị null trong cột '{columns}': {count_null_values}")

Số giá trị null trong cột '['Tên đăng bán', 'Giá bán', 'Hãng', 'Dòng xe', 'Năm sản xuất', 'Số Km đã đi', 'Tình trạng', 'Hộp số', 'Nhiên liệu', 'Xuất xứ', 'Kiểu dáng', 'Số chỗ', 'Chính sách bảo hành', 'Trọng lượng', 'Trọng tải']': Tên đăng bán            0
Giá bán                 0
Hãng                    0
Dòng xe                 0
Năm sản xuất            0
Số Km đã đi            22
Tình trạng              0
Hộp số                  1
Nhiên liệu              1
Xuất xứ                 0
Kiểu dáng              62
Số chỗ                 64
Chính sách bảo hành     0
Trọng lượng             0
Trọng tải               0
dtype: int64


In [312]:
# Xóa các dòng có giá trị NaN
data = data.dropna()

# Điền lại số thứ tự từ đầu
data = data.reset_index(drop=True)

In [313]:
data

,Tên đăng bán,Giá bán,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Chính sách bảo hành,Trọng lượng,Trọng tải
0,Tên đăng bán,Giá bán,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Chính sách bảo hành,Trọng lượng,Trọng tải
1,Mazda cx5 2.0 2021 màu đỏ bản Premium,755.000.000 đ,Mazda,CX 5,2021,38000,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,5,Bảo hành hãng,> 1 tấn,> 2 tấn
2,💥Mercedes c200 sx 2017 2.0 AT cao cấp od 6v km 💥,785.000.000 đ,Mercedes Benz,C Class,2017,60000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn
3,XE Range Rover HSE lên Model 2020 bản 5 ghế,1.999.000.000 đ,LandRover,Range Rover,2013,2,Đã sử dụng,Tự động,Xăng,Nước khác,SUV / Cross over,8,Bảo hành hãng,> 1 tấn,> 2 tấn
4,Bán xe Camry 2.0G 2021 Thái Lan. Trả góp 300,885.000.000 đ,Toyota,Camry,2021,28000,Đã sử dụng,Tự động,Xăng,Thái Lan,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,Hyundai Accent 2022 bản đủ ATH biển Hà Nội 1 chủ,485.000.000 đ,Hyundai,Accent,2022,20000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,2,Bảo hành hãng,> 1 tấn,> 2 tấn
498,ban Suzuki xl 7 sx 2022,529.000.000 đ,Suzuki,XL 7,2022,30000,Đã sử dụng,Tự động,Xăng,Ấn Độ,Minivan (MPV),7,Bảo hành hãng,> 1 tấn,> 2 tấn
499,Toyota Fortuner 2013 máy xăng số tự động,435.000.000 đ,Toyota,Fortuner,2013,110000,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,7,Bảo hành hãng,> 1 tấn,> 2 tấn
500,Vinfast Fadil bản plus sx cuối 2020 Full đồ,340.000.000 đ,Vinfast,Fadil,2020,24256,Đã sử dụng,Tự động,Xăng,Việt Nam,Hatchback,5,Bảo hành hãng,> 1 tấn,> 2 tấn


Bộ dữ liệu ban đầu có 590 data sau khi lọc những data có giá trị null thì còn lại 501 data

In [314]:
duplicate_rows = data[data.duplicated(keep='first')]

# In ra các dòng bị trùng
print("Các dòng trùng nhau:")
print(duplicate_rows)

# Xóa các dòng trùng nhau và giữ lại dòng đầu tiên
data = data.drop_duplicates(keep='first')

# Điền lại số thứ tự từ đầu
data = data.reset_index(drop=True)

Các dòng trùng nhau:
                                          Tên đăng bán           Giá bán  \
57     Mercedes Benz GLC200 2022 siêu lướt BH đến 2025  1.569.000.000 đ    
58      Mercedes C200 Sản Xuất 2016 Trắng Nội Thất Kem    739.000.000 đ    
78   Mercedes C300 AMG Model 2019 Nội Thất Nâu Bank...  1.099.000.000 đ    
461       Bán xe KIA K3, bản cao cấp 2.0. Đi hơn 9 vạn    390.000.000 đ    

              Hãng    Dòng xe Năm sản xuất Số Km đã đi  Tình trạng   Hộp số  \
57   Mercedes Benz  GLC Class         2022        5000  Đã sử dụng  Tự động   
58   Mercedes Benz    C Class         2016       73000  Đã sử dụng  Tự động   
78   Mercedes Benz    C Class         2018       43000  Đã sử dụng  Tự động   
461            Kia         K3         2015       97000  Đã sử dụng  Tự động   

    Nhiên liệu        Xuất xứ         Kiểu dáng Số chỗ Chính sách bảo hành  \
57        Xăng  Đang cập nhật  SUV / Cross over      5       Bảo hành hãng   
58        Xăng       Việt Nam             Seda

In [315]:
data

,Tên đăng bán,Giá bán,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Chính sách bảo hành,Trọng lượng,Trọng tải
0,Tên đăng bán,Giá bán,Hãng,Dòng xe,Năm sản xuất,Số Km đã đi,Tình trạng,Hộp số,Nhiên liệu,Xuất xứ,Kiểu dáng,Số chỗ,Chính sách bảo hành,Trọng lượng,Trọng tải
1,Mazda cx5 2.0 2021 màu đỏ bản Premium,755.000.000 đ,Mazda,CX 5,2021,38000,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,5,Bảo hành hãng,> 1 tấn,> 2 tấn
2,💥Mercedes c200 sx 2017 2.0 AT cao cấp od 6v km 💥,785.000.000 đ,Mercedes Benz,C Class,2017,60000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn
3,XE Range Rover HSE lên Model 2020 bản 5 ghế,1.999.000.000 đ,LandRover,Range Rover,2013,2,Đã sử dụng,Tự động,Xăng,Nước khác,SUV / Cross over,8,Bảo hành hãng,> 1 tấn,> 2 tấn
4,Bán xe Camry 2.0G 2021 Thái Lan. Trả góp 300,885.000.000 đ,Toyota,Camry,2021,28000,Đã sử dụng,Tự động,Xăng,Thái Lan,Sedan,5,Bảo hành hãng,> 1 tấn,> 2 tấn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,Hyundai Accent 2022 bản đủ ATH biển Hà Nội 1 chủ,485.000.000 đ,Hyundai,Accent,2022,20000,Đã sử dụng,Tự động,Xăng,Đang cập nhật,Sedan,2,Bảo hành hãng,> 1 tấn,> 2 tấn
494,ban Suzuki xl 7 sx 2022,529.000.000 đ,Suzuki,XL 7,2022,30000,Đã sử dụng,Tự động,Xăng,Ấn Độ,Minivan (MPV),7,Bảo hành hãng,> 1 tấn,> 2 tấn
495,Toyota Fortuner 2013 máy xăng số tự động,435.000.000 đ,Toyota,Fortuner,2013,110000,Đã sử dụng,Tự động,Xăng,Việt Nam,SUV / Cross over,7,Bảo hành hãng,> 1 tấn,> 2 tấn
496,Vinfast Fadil bản plus sx cuối 2020 Full đồ,340.000.000 đ,Vinfast,Fadil,2020,24256,Đã sử dụng,Tự động,Xăng,Việt Nam,Hatchback,5,Bảo hành hãng,> 1 tấn,> 2 tấn


Có 4 dòng trùng nhau sau khi đã lọc những dòng có giá trị null

In [316]:
filtered_data = data[data['Xuất xứ'] == 'Đang cập nhật']

# In ra các dòng dữ liệu đã lọc
print(filtered_data)

                                          Tên đăng bán           Giá bán  \
2     💥Mercedes c200 sx 2017 2.0 AT cao cấp od 6v km 💥    785.000.000 đ    
12        Mercedes Benz E Class E180 2022 lướt 7000 km  1.590.000.000 đ    
14   ✅ Honda CRV 1.5Turbo bản L NHẬP KHẨU nguyên chiếc    755.000.000 đ    
15   Audi A6 sx 2015 đi 6v5 mile giá mềm cho ae chơ...    815.000.000 đ    
16     Mercedes Benz GLC200 2022 siêu lướt BH đến 2025  1.569.000.000 đ    
..                                                 ...               ...   
490   Mercedes C Class C200 Avantgarde sx 2022 odo 1v2  1.319.000.000 đ    
491    🚘💥Xe Mercedes GLK 300 4MATIC sx 2009 odo 13v 💥🚘    355.000.000 đ    
492  Peugeot 208 Model 2014 , chạy 48.000 KM cực đẹp ạ    340.000.000 đ    
493   Hyundai Accent 2022 bản đủ ATH biển Hà Nội 1 chủ    485.000.000 đ    
497   Kia Seltos 1.4AT đăng ký lần đầu T1/2021 biển HN    590.000.000 đ    

              Hãng    Dòng xe Năm sản xuất Số Km đã đi  Tình trạng   Hộp số  \
2    Mer

Có 135 cột Xuất xứ có giá trị là đang cập nhật (không rõ nguồn gốc, xuất xứ)

In [319]:
with open('output.csv', 'w', newline='', encoding='utf-8-sig') as file:
    csv_writer = csv.writer(file, delimiter=';')
    # Ghi tiêu đề (nếu bạn có tiêu đề)
    csv_writer.writerow(data.columns)

    # Ghi dữ liệu từ DataFrame
    for index, row in data.iterrows():
        csv_writer.writerow(row)

In [320]:
files.download('output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>